<a href="https://colab.research.google.com/github/puneethkumarvydya/Capstone-Project/blob/Ganesh's-Branch/Capstone_Project_AIML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Pneumonia detection from chest X-Ray images 

In [ ]:
ganesh prasad